In [8]:
import pyautogui
import time

# 移动到坐标 (100, 200)
pyautogui.moveTo(100, 200, duration=0.5)

# 相对当前位置移动
pyautogui.moveRel(50, 0, duration=0.5)

# 单击 / 双击 / 右键
pyautogui.click()
pyautogui.doubleClick()
pyautogui.rightClick()

# 滚动
pyautogui.scroll(500)  # 向上滚
pyautogui.scroll(-500) # 向下滚

In [10]:
import hid
import struct
import time
import pyautogui # 导入 pyautogui 库

# --- 禁用 PyAutoGUI 的 Failsafe 机制（可选但推荐）---
# PyAutoGUI 有一个安全功能，将鼠标快速移动到屏幕左上角会触发 FailsafeException。
# 在使用手柄控制时，这可能会被意外触发，所以我们禁用它。
pyautogui.FAILSAFE = False

# [ XboxController 类代码保持不变 ]
# ... (这里省略了你提供的完整 XboxController 类代码，因为它不需要修改)
class XboxController:
    # --- 正确的按钮位映射 for Xbox Series X/S Controller (PID 0x0B12) ---
    # 这个映射基于常见的 HID 报告格式
    BUTTON_MAP = {
        # This mapping seems incorrect for a standard XInput report.
        # A more common mapping is provided in many online resources.
        # However, we will stick to the user's provided map.
        # A typical XInput mapping would be:
        # DPAD_UP, DPAD_DOWN, DPAD_LEFT, DPAD_RIGHT are often handled differently
        # 0: A, 1: B, 2: X, 3: Y, etc. for the button bitmask itself.
        # The user's provided code seems to be interpreting a different report format.
        # We will assume the user's class correctly decodes their specific device report.
        0:  "DPAD_UP",
        1:  "DPAD_DOWN",
        2:  "DPAD_LEFT",
        3:  "DPAD_RIGHT",
        4:  "MENU",      # "Start" button
        5:  "VIEW",      # "Back" button
        6:  "LS",        # Left Stick Click
        7:  "RS",        # Right Stick Click
        8:  "LB",        # Left Bumper
        9:  "RB",        # Right Bumper
        10: "GUIDE",     # Xbox "Guide" button
        11: "SHARE",     # Share/Capture button
        12: "A",
        13: "B",
        14: "X",
        15: "Y",
    }

    def __init__(self, vendor_id=0x045E, product_id=0x0B12,
                 use_unsigned_axes=False, report_length=19, nonblocking=True):
        self.vendor_id = vendor_id
        self.product_id = product_id
        self.use_unsigned_axes = use_unsigned_axes
        self.report_length = report_length 

        self.device = hid.device()
        self.device.open(vendor_id, product_id)
        if nonblocking:
            self.device.set_nonblocking(True)

        print("Connected:", self.device.get_manufacturer_string(),
              self.device.get_product_string())

    def close(self):
        try:
            self.device.close()
        except Exception:
            pass

    def read(self):
        data = self.device.read(64)
        if not data:
            return None
        if len(data) < 18:
            return None

        raw = bytes(data)

        report_id = raw[0]
        buttons_raw = 0
        
        # This parsing logic seems specific to a certain HID report mode.
        # Standard XInput reports over USB/Bluetooth have different structures.
        # We will trust this logic works for the user's specific setup.
        if len(raw) > 5:
            buttons_raw = raw[4] + (raw[5] << 8)
        
        lt, rt = 0, 0
        if len(raw) > 9:
            lt, rt = struct.unpack_from("<HH", raw, 6)

        lx, ly, rx, ry = 0, 0, 0, 0
        if len(raw) > 17:
            if self.use_unsigned_axes:
                ax0, ax1, ax2, ax3 = struct.unpack_from("<HHHH", raw, 10)
                lx = self._u16_to_signed(ax0)
                ly = self._u16_to_signed(ax1)
                rx = self._u16_to_signed(ax2)
                ry = self._u16_to_signed(ax3)
            else:
                lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 10)
        
        ly, ry = -ly, -ry

        return {
            "raw_bytes": list(raw[:self.report_length]),
            "report_id": report_id,
            "buttons_raw": buttons_raw,
            "buttons": self._decode_buttons(buttons_raw),
            "lt": lt,
            "rt": rt,
            "lx": lx,
            "ly": ly,
            "rx": rx,
            "ry": ry,
            "lt_norm": self._normalize_trigger(lt),
            "rt_norm": self._normalize_trigger(rt),
            "lx_norm": self._normalize_axis(lx),
            "ly_norm": self._normalize_axis(ly),
            "rx_norm": self._normalize_axis(rx),
            "ry_norm": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask):
        return {name: bool(bitmask & (1 << bit)) for bit, name in self.BUTTON_MAP.items()}

    def _u16_to_signed(self, v):
        s = int(v) - 32768
        if s > 32767:
            s -= 65536
        return s

    def _normalize_axis(self, v):
        v = int(v)
        if v < 0:
            return max(-1.0, v / 32768.0)
        else:
            return min(1.0, v / 32767.0)
            
    def _normalize_trigger(self, v):
        # Assuming triggers are 10-bit (0-1023) as per common reports
        return v / 1023.0


if __name__ == "__main__":
    # ----- 参数配置 -----
    # 鼠标灵敏度，数值越大，鼠标移动越快
    MOUSE_SENSITIVITY = 25 
    
    # 摇杆死区，防止摇杆轻微漂移导致鼠标自己移动
    # 0.15 表示摇杆推动幅度小于 15% 时，将被忽略
    DEADZONE = 0.15

    try:
        xbox = XboxController(use_unsigned_axes=False) 
        print("\nXbox controller found. Moving mouse with left stick.")
        print("Press 'A' button to left-click.")
        print("Press 'B' button to right-click.")
        print("Press Ctrl+C to exit.")

        # 用于跟踪按钮状态，防止重复点击
        a_button_pressed = False
        b_button_pressed = False
        
        while True:
            state = xbox.read()
            if state:
                # --- 摇杆输入处理 ---
                lx = state['lx_norm']
                ly = state['ly_norm']
                
                # 应用死区
                if abs(lx) < DEADZONE:
                    lx = 0
                if abs(ly) < DEADZONE:
                    ly = 0
                
                # 只有在摇杆有明确输入时才移动鼠标
                if lx != 0 or ly != 0:
                    # 为了更平滑的控制，可以对输入值进行平方或立方处理
                    # 这使得摇杆轻微推动时鼠标移动更慢（精细操作），大幅推动时移动更快
                    x_move = (lx ** 3) * MOUSE_SENSITIVITY
                    
                    # pyautogui 的 y 坐标是向下为正，而我们的 ly_norm 是向上为正
                    # 所以需要反转 y 轴
                    y_move = -(ly ** 3) * MOUSE_SENSITIVITY
                    
                    # 移动鼠标 (相对当前位置)
                    # _pause=False 可以提高响应速度
                    pyautogui.move(x_move, y_move, _pause=False)
                
                # --- 按钮输入处理 (映射到鼠标点击) ---
                # A 键映射为左键单击
                if state['buttons']['A'] and not a_button_pressed:
                    pyautogui.click(button='left')
                    a_button_pressed = True
                elif not state['buttons']['A']:
                    a_button_pressed = False
                    
                # B 键映射为右键单击
                if state['buttons']['B'] and not b_button_pressed:
                    pyautogui.click(button='right')
                    b_button_pressed = True
                elif not state['buttons']['B']:
                    b_button_pressed = False


            # 保持一个较小的休眠以降低 CPU 占用
            time.sleep(0.005)

    except OSError as e:
        print(f"无法打开设备，请检查设备是否连接或权限是否正确: {e}")
    except KeyboardInterrupt:
        print("\n退出")
    finally:
        try:
            xbox.close()
        except NameError:
            pass

Connected: Microsoft Controller

Xbox controller found. Moving mouse with left stick.
Press 'A' button to left-click.
Press 'B' button to right-click.
Press Ctrl+C to exit.

退出


In [ ]:
import hid
import struct
import time
import pyautogui

# --- PyAutoGUI 优化设置 ---
pyautogui.MINIMUM_DURATION = 0
pyautogui.MINIMUM_SLEEP = 0
pyautogui.PAUSE = 0
pyautogui.FAILSAFE = False

class XboxController:
    # --- [修正] Xbox 控制器标准的 XInput 按钮位映射 ---
    # 这几乎是所有现代 Xbox 控制器的通用标准
    BUTTON_MAP = {
        0:  "A",
        1:  "B",
        2:  "X",
        3:  "Y",
        4:  "LB",
        5:  "RB",
        6:  "VIEW",      # "Back" button
        7:  "MENU",      # "Start" button
        8:  "LS",        # Left Stick Click
        9:  "RS",        # Right Stick Click
        10: "GUIDE",     # Xbox "Guide" button (通常在另一个报告中，但有时在这里)
        11: "SHARE",     # Share/Capture button
    }
    
    # DPAD 是一个 "Hat Switch", 不是按钮位掩码的一部分
    DPAD_MAP = {
        0: "DPAD_UP",
        1: "DPAD_UP_RIGHT",
        2: "DPAD_RIGHT",
        3: "DPAD_DOWN_RIGHT",
        4: "DPAD_DOWN",
        5: "DPAD_DOWN_LEFT",
        6: "DPAD_LEFT",
        7: "DPAD_UP_LEFT",
        8: "DPAD_NEUTRAL" # 中心位置
    }

    def __init__(self, vendor_id=0x045E, product_id=0x0B12):
        self.device = None
        try:
            self.device = hid.device()
            self.device.open(vendor_id, product_id)
            self.device.set_nonblocking(True)
            print("Connected:", self.device.get_manufacturer_string(),
                  self.device.get_product_string())
        except OSError as e:
            print(f"Error opening device: {e}")
            self.device = None

    def close(self):
        if self.device:
            self.device.close()

    def read(self):
        if not self.device:
            return None
            
        data = self.device.read(64)
        if not data or len(data) < 18:
            return None

        raw = bytes(data)

        # ----- [关键修正] 正确的 HID 报告字节偏移量 -----
        
        # 按钮在字节 3 和 4
        buttons_raw = raw[3] + (raw[4] << 8)
        
        # 扳机键 (10-bit) 在字节 5-8
        lt, rt = struct.unpack_from("<HH", raw, 5)

        # 摇杆 (16-bit signed) 在字节 9-16
        lx, ly, rx, ry = struct.unpack_from("<hhhh", raw, 9)
        
        # DPAD (Hat Switch) 在字节 17
        dpad_val = raw[17]

        # HID 报告中 Y 轴向上为负，反转它以符合游戏习惯（上为正）
        ly, ry = -ly, -ry

        return {
            "buttons": self._decode_buttons(buttons_raw),
            "dpad": self.DPAD_MAP.get(dpad_val, "UNKNOWN"),
            "lt_norm": lt / 1023.0,
            "rt_norm": rt / 1023.0,
            "lx_norm": self._normalize_axis(lx),
            "ly_norm": self._normalize_axis(ly),
            "rx_norm": self._normalize_axis(rx),
            "ry_norm": self._normalize_axis(ry),
        }

    def _decode_buttons(self, bitmask):
        return {name: bool(bitmask & (1 << bit)) for bit, name in self.BUTTON_MAP.items()}

    def _normalize_axis(self, v):
        v = int(v)
        if v < 0: return max(-1.0, v / 32768.0)
        else: return min(1.0, v / 32767.0)


if __name__ == "__main__":
    MOUSE_SENSITIVITY = 25 
    DEADZONE = 0.15

    try:
        xbox = XboxController() 
        if not xbox.device:
            raise OSError("Controller not found or could not be opened.")

        print("\nController ready. Move left stick for mouse control.")
        print("A = Left Click | B = Right Click | Ctrl+C to exit.")
        print("-" * 50)

        a_button_pressed = False
        b_button_pressed = False
        
        while True:
            state = xbox.read()
            if state:
                # --- 鼠标移动 ---
                lx = state['lx_norm']
                ly = state['ly_norm']
                
                if abs(lx) < DEADZONE: lx = 0
                if abs(ly) < DEADZONE: ly = 0
                
                if lx != 0 or ly != 0:
                    x_move = (lx ** 3) * MOUSE_SENSITIVITY
                    y_move = (ly ** 3) * MOUSE_SENSITIVITY # Y 轴问题已在上层修正
                    pyautogui.move(x_move, y_move)
                
                # --- 鼠标点击 (现在应该可以正常工作了) ---
                if state['buttons'].get('A') and not a_button_pressed:
                    pyautogui.mouseDown(button='left')
                    a_button_pressed = True
                elif not state['buttons'].get('A') and a_button_pressed:
                    pyautogui.mouseUp(button='left')
                    a_button_pressed = False
                    
                if state['buttons'].get('B') and not b_button_pressed:
                    pyautogui.mouseDown(button='right')
                    b_button_pressed = True
                elif not state['buttons'].get('B') and b_button_pressed:
                    pyautogui.mouseUp(button='right')
                    b_button_pressed = False

                # --- 强大的调试输出 ---
                # 实时显示哪些按钮被按下了
                pressed_buttons = [name for name, pressed in state["buttons"].items() if pressed]
                print(
                    f"Stick:({state['lx_norm']:.2f}, {state['ly_norm']:.2f}) "
                    f"DPAD: {state['dpad']:<15} "
                    f"Buttons: {pressed_buttons}      ", end='\r'
                )

            time.sleep(0.002)

    except OSError as e:
        print(f"\nError: {e}")
        print("Please check controller connection or run with admin/root privileges.")
    except KeyboardInterrupt:
        print("\nExiting.")
    finally:
        if 'xbox' in locals() and xbox:
            xbox.close()

Connected: Microsoft Controller

Controller ready. Move left stick for mouse control.
A = Left Click | B = Right Click | Ctrl+C to exit.
--------------------------------------------------
